### Master oficial en Big Data & Data Science - Universidad Internacional de Valencia (VIU)

<font color="darkblue">

-   **Alumno:** Franco Rojas Yucra
-   **TFM:** Sistema para la busqueda de conocimientos en textos de documentos multimedia
-   **Email:** franco.rojasyucra@alumnos.viu.es
-   **Fecha:** Octubre de 2023
<font color="black">

## 03 Exploración y preparación de corpus

Librerías generales

In [1]:
import pandas as pd
import string
import re
import nltk ## Funciones de Text mining
from langdetect import detect #detectar idioma
import requests
import hashlib

C:\Users\Franco\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#nltk.download() ## Solo se ejecuta la primera vez que se utiliza nltk

In [3]:
pd.set_option('display.max_columns',None) #motrar todas las columnas

Lista de documentos multimedia de Youtube

In [4]:
df_listadoc=pd.read_excel('lista_documentos.xlsx')

In [5]:
df_listadoc.head()

,id_doc,tema,titulo_doc,url_doc,duracion
0,DOC0001,Internet de las cosas,01 - Curso Internet de las Cosas - Introducción,https://www.youtube.com/watch?v=5-MQj1kD9-I,8:44
1,DOC0002,Internet de las cosas,02 - Curso Internet de las Cosas - Laboratorio...,https://www.youtube.com/watch?v=GExTkcH5pTg,11:23
2,DOC0003,Introduccion IA,¿QUÉ ES LA INTELIGENCIA ARTIFICIAL? | 01 Intel...,https://www.youtube.com/watch?v=KmhLHDONTXY,12:10
3,DOC0004,Introduccion IA,¿QUÉ ES LA MACHINE LEARNING? | 02 Inteligencia...,https://www.youtube.com/watch?v=bFQmEx9T29U,14:19
4,DOC0005,Introduccion IA,¿QUÉ ES DEEP LEARNING? | 03 Inteligencia Artif...,https://www.youtube.com/watch?v=xuSnmWZagN8,11:40


#### Reuniendo textos en una tabla

In [6]:
#Acumulando textos en una sola tabla por id_doc
df_transcrip=pd.DataFrame()
for id_doc,tema in zip(list(df_listadoc.id_doc),list(df_listadoc.tema)):
    f = open("transcripcion_whisper/"+id_doc+"_text.txt",encoding='utf8')
    df_trtmp=pd.DataFrame({'id_doc':[id_doc],'tema':tema,'transcription':f.readlines()[0].strip()})
    f.close()
    df_transcrip=pd.concat([df_transcrip,df_trtmp]).reset_index(drop=True)

In [7]:
df_transcrip.head()

,id_doc,tema,transcription
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern..."
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...
3,DOC0004,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...
4,DOC0005,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...


In [8]:
df_transcrip.to_csv('transformed_corpus/conjunto_transcripciones_ini.csv',index=False,sep='|')

## Filtros de calidad

### 1. Retiro de documentos con poca cantidad de palabras

In [9]:
# Calculando cantidad de tokens por documento
df_transcrip['tokens']=df_transcrip['transcription'].apply(lambda x : len(x.split()))

In [10]:
df_transcrip.head(3)

,id_doc,tema,transcription,tokens
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern...",1350
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...,1605
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,1671


In [11]:
#Cantidad inicial de palabras/tokens
df_transcrip.tokens.sum()

257067

In [12]:
df_transcrip.tokens.describe()

count     150.000000
mean     1713.780000
std       921.216382
min        20.000000
25%       993.500000
50%      1640.000000
75%      2277.750000
max      4057.000000
Name: tokens, dtype: float64

In [13]:
df_transcrip[df_transcrip['tokens']<100]

,id_doc,tema,transcription,tokens
21,DOC0022,Datalakes,As you wrap up the process of designing the pr...,20


*Se retira documento con 20 tokens, y se observa tambien que esta en ingles.*

In [14]:
df_transcrip=df_transcrip[df_transcrip['tokens']>=100].reset_index(drop=True)

In [15]:
#Cantidad de tokens despues de retiro
df_transcrip.tokens.sum()

257047

### 2. Verificación de idioma de documentos

Función para separar texto en frases

In [16]:
#Se considera frases de 5 palabras para evaluar idioma
def dividir_en_frases(text):   
    frases=[]; i = 0;
    while i < len(text.split()):
        if i >= len(text.split()): break
        frases.append(" ".join(text.split()[i:i+5]))
        i+=5
    return frases

In [17]:
signos_numer=string.punctuation+'?¿´“'+'1234567890'
signos_numer

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~?¿´“1234567890'

Función para calcular porcentaje de frases en idioma español del total de frases

In [18]:
def pct_español_frases(text):
    lst_id=[]
    text2=re.sub('[%s]' % re.escape(signos_numer), '', text)
    for x in dividir_en_frases(text2):
        idio_det=detect(x)
        lst_id.append(idio_det)
    return len([x for x in lst_id if x=='es'])/len(lst_id)

Aplicando función de calculo de porcentaje de idioma español

In [19]:
%%time
df_transcrip['pct_esp_transc']=df_transcrip['transcription'].apply(lambda x : pct_español_frases(x))

In [22]:
df_transcrip.head()

,id_doc,tema,transcription,tokens,pct_esp_transc
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern...",1350,0.940741
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...,1605,0.896552
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,1671,0.918919
3,DOC0004,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,2086,0.858173
4,DOC0005,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,1693,0.866864


In [23]:
df_transcrip.pct_esp_transc.describe()

count    149.000000
mean       0.876526
std        0.042524
min        0.709677
25%        0.853116
50%        0.881720
75%        0.908257
max        0.963211
Name: pct_esp_transc, dtype: float64

Se evalua aquellos documentos con menos de 75% de frases de idioma español

In [24]:
df_transcrip[df_transcrip['pct_esp_transc']<0.75]

,id_doc,tema,transcription,tokens,pct_esp_transc
19,DOC0020,Datalakes,"¿Cuentas con una gran cantidad de datos, pero ...",154,0.709677
118,DOC0120,Data Storytelling,de la historia. Por qué ahora todos quieren me...,1380,0.717391


In [25]:
df_transcrip['transcription'][19]

'¿Cuentas con una gran cantidad de datos, pero aún no sabes cómo transformarlos en valor? Sumérgete en el lago de datos con Plug & Play Data Lake. Desde Nuviral, comandamos la expedición. Plug & Play Data Lake permite etiquetar, buscar, compartir, transformar, analizar y administrar fácilmente sus conjuntos de datos en una empresa o con usuarios externos. ¿Por qué es necesario que todas las empresas cuenten con esta solución? Porque exporta y almacena datos de forma segura, a gran escala y bajo costo. Realiza transformaciones de datos de manera sencilla. Proporciona data de calidad desde un solo almacén. Garantiza la disponibilidad y el análisis en todo momento. Permite el acceso y análisis de datos desde una misma fuente de información para todos los usuarios. Entre tres paquetes diferentes, puedes elegir el que mejor se adapte a tu empresa. Bronze, Silver o Gold. El momento es ahora. Sumérgete hoy mismo en este lago de oportunidades.'

In [26]:
df_transcrip['transcription'][118][:1000]

'de la historia. Por qué ahora todos quieren meter historias. Yo soy Rodrigo Marquez y soy un especialista en visualización de datos decía la escritora Maya ángelo las personas van a olvidar lo que dijiste pero nunca van a olvidar cómo las sientes. Acá de pasar. Y ahora yo estoy en la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia de la noticia 

*El DOC0020 se retira por tener muchas palabras tecnicas en ingles y por tener pocos tokens. El DOC0120 se retira por tener palabras repetidas, errores en transcripción.*

In [27]:
df_transcrip=df_transcrip[df_transcrip['pct_esp_transc']>=0.75].reset_index(drop=True)

In [28]:
#Cantidad de tokens despues de retiro
df_transcrip.tokens.sum()

255513

### 3. Retiro de malas palabras (bad_words)

Para este filtro se descarga un corpus de bad_words disponibles en Github.

In [29]:
response = requests.get('https://github.com/EddieSharp/Insultos/blob/master/diccionario.txt')
str_url_imp=response.text
#Realizando busqueda de lista de badwords en el request
lst_url_imp=str_url_imp[str_url_imp.find('"A')+1:str_url_imp.find('"],"')-1].split('","')

Función para estandarizar y limpieza general de textos

In [30]:
def cleaning_text(text):
    x = text.lower()
    x = re.sub(r'[0-9]', "", x)
    x = re.sub('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~?¿´“”'), '', x)
    x = x.translate(str.maketrans('àèìòùáéíóúüñÁÉÍÓÚÜÑ','aeiouaeiouunAEIOUUN'))
    return x

In [32]:
bad_words=[cleaning_text(str(pal)) for pal in lst_url_imp]
len(bad_words)

878

In [33]:
#Primeras 15 bad_words
bad_words[:15]

['abadol',
 'abejarruca',
 'abejarruco',
 'aberracion',
 'aberracion',
 'aberrante',
 'ablanda brevas',
 'abollao',
 'aborto',
 'abrazafarolas',
 'abrillantaglandes',
 'abulto',
 'adan',
 'afilasables',
 'aguafiestas']

Función para remover bad_words

In [34]:
def remove_badwords(text):
    text_sep_filt=[]
    text_sep=re.findall(r"\w+|\W+", text)
    text_sep_clean=[cleaning_text(str(pal)) for pal in text_sep]
    for i in range(len(text_sep_clean)):
        if text_sep_clean[i] not in bad_words: 
                text_sep_filt.append(text_sep[i])
    text_sep_filt1="".join(text_sep_filt)
    text_sep_filt2=[x for x in text_sep_filt1.split() if x not in string.punctuation+'?¿´“']
    return " ".join(text_sep_filt2)

Aplicando función de retiro de bad_words a todo el corpus

In [35]:
%%time
df_transcrip['transcription_filt1']=df_transcrip['transcription'].apply(lambda x : remove_badwords(x))

CPU times: total: 9.58 s
Wall time: 9.61 s


In [36]:
# Calculando cantidad de tokens por documento
df_transcrip['tokens_filt1']=df_transcrip['transcription_filt1'].apply(lambda x : len(x.split()))

In [37]:
df_transcrip[['tokens','tokens_filt1']].sum()

tokens          255513
tokens_filt1    255474
dtype: int64

*Se han retirado 39 tokens detectados como bad_words.*

In [38]:
df_transcrip.head(3)

,id_doc,tema,transcription,tokens,pct_esp_transc,transcription_filt1,tokens_filt1
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern...",1350,0.940741,"Bienvenidos a nuestro canal de YouTube, Intern...",1350
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...,1605,0.896552,Bienvenidos al curso Internet de las Cosas. Mi...,1605
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,1671,0.918919,Hola a todos y bienvenidos a un nuevo video en...,1671


## Deduplicados de textos

Función para separar texto en frases

In [39]:
#Se considera frases de 10 palabras para evaluar deduplicacion
def dividir_en_frases(text):   
    frases=[]; i = 0;
    while i < len(text.split()):
        if i >= len(text.split()): break
        frases.append(" ".join(text.split()[i:i+10]))
        i+=10
    return frases

Función para retirar frases duplicadas

In [40]:
def deduplicate_text_by_hash(text):
    frases=dividir_en_frases(text)
    hashes = set()
    unique_texts = []
    
    for text in frases:
        hash = hashlib.sha256(text.encode()).hexdigest()
        if hash not in hashes:
            hashes.add(hash)
            unique_texts.append(text)
    
    return " ".join(unique_texts)

Aplicando función de duplicados a todo el corpus

In [41]:
%%time
df_transcrip['transcription_filt2']=df_transcrip['transcription_filt1'].apply(lambda x : deduplicate_text_by_hash(x))

CPU times: total: 12.7 s
Wall time: 12.8 s


In [42]:
# Calculando cantidad de tokens por documento
df_transcrip['tokens_filt2']=df_transcrip['transcription_filt2'].apply(lambda x : len(x.split()))

In [43]:
df_transcrip[['tokens','tokens_filt1','tokens_filt2']].sum()

tokens          255513
tokens_filt1    255474
tokens_filt2    255414
dtype: int64

In [44]:
df_transcrip[df_transcrip['tokens_filt1']!=df_transcrip['tokens_filt2']]

,id_doc,tema,transcription,tokens,pct_esp_transc,transcription_filt1,tokens_filt1,transcription_filt2,tokens_filt2
7,DOC0008,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...,2083,0.927885,Hola a todos y bienvenidos a un nuevo video en...,2083,Hola a todos y bienvenidos a un nuevo video en...,2073
31,DOC0034,Internet de las cosas,"Hola a todos, continuando con la serie de vide...",999,0.815000,"Hola a todos, continuando con la serie de vide...",999,"Hola a todos, continuando con la serie de vide...",989
70,DOC0073,NLP e IA,La inteligencia artificial cada vez avanza más...,2588,0.837255,La inteligencia artificial cada vez avanza más...,2588,La inteligencia artificial cada vez avanza más...,2568
72,DOC0075,NLP e IA,Hasta este momento ya tenemos muchas opciones ...,1456,0.862543,Hasta este momento ya tenemos muchas opciones ...,1456,Hasta este momento ya tenemos muchas opciones ...,1446
81,DOC0084,LLM,Muchos hemos oído hablar del término token cua...,2336,0.885714,Muchos hemos oído hablar del término token cua...,2336,Muchos hemos oído hablar del término token cua...,2326


#### Guardando corpus final luego de filtros de calidad y deduplicación

In [45]:
df_transcrip[['id_doc','tema','transcription_filt2']].head(3)

,id_doc,tema,transcription_filt2
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern..."
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...


In [46]:
df_transcrip[['id_doc','tema','transcription_filt2']].to_csv('transformed_corpus/conjunto_transcripciones_fin.csv',
                                                             index=False,sep='|')